# How to create a model merging chat?


We will first define the list of models we want to use and merge.

now we will define one chat object, we will then clone it for each model and call it.
Then we will take the last element of .h in the history list and prodive them all with the common history to a
ask() function.

In [1]:
import onetokenpy as ot
from IPython.display import Markdown
from copy import deepcopy
def prompt_and_merge_chat(prompt, chat, list_of_models):
    responses = []
    for model in list_of_models:
        tmp_chat_branch = deepcopy(chat)
        tmp_chat_branch.model = model
        res = ot.contents(tmp_chat_branch(prompt))
        responses.append(res)
    
    merged_response = chat(
        f"""You have been provided with a set of responses from various models to the latest user query: "{prompt}"
Your task is to synthesize these responses into a single, high-quality response. It is crucial to critically evaluate the information provided in these responses, recognizing that some of it may be biased or incorrect. Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.
Responses from models: {responses}

Do not refer to the models in your response, just use then for crafting the best answer to the user query.""")
    chat.h[-2] = {"role":"user", "content":prompt}
    chat.h[-1] = {"role":"assistant", "content":ot.contents(merged_response)}
    return Markdown(chat.h[-1]["content"])

In [64]:
list_of_models = [
    "anthropic/claude-3.5-haiku",
    "openai/gpt-4.1",
    "x-ai/grok-3-mini-beta",
    "google/gemini-2.5-flash-preview"
]

chat = ot.new_chat(model="x-ai/grok-3-mini-beta")
prompt_and_merge_chat("Tell me a joke", chat, list_of_models)


Sure, I'd be happy to share a joke with you!

Why don't scientists trust atoms? Because they make up everything!

I hope that brought a smile to your face. If you'd like another one, just let me know! 😊

In [65]:
chat.h

[{'role': 'user', 'content': 'Tell me a joke'},
 {'role': 'assistant',
  'content': "Sure, I'd be happy to share a joke with you!\n\nWhy don't scientists trust atoms? Because they make up everything!\n\nI hope that brought a smile to your face. If you'd like another one, just let me know! 😊"}]

In [3]:
from onetokenpy import new_chat, Attachments
list_of_models = [
    "anthropic/claude-3.5-haiku",
    "openai/gpt-4.1",
    "x-ai/grok-3-mini-beta",
    "google/gemini-2.5-flash-preview"
]
chat = new_chat(attach=Attachments("https://answerdotai.github.io/cosette/llms.txt"))
prompt_and_merge_chat("What is cosette?", chat, list_of_models)

Model anthropic/claude-3.5-haiku response: According to the text, Cosette is a Python library that wraps OpenAI's API to provide a higher-level interface for creating AI applications. Key features include:

1. Automating common patterns while maintaining full control
2. Providing stateful chat functionality
3. Handling images
4. Streamlining tool use

Some important notes about Cosette:
- It requires setting the `OPENAI_API_KEY` environment variable
- It works with various OpenAI models like 'gpt-4o', 'gpt-4-turbo', 'gpt-3.5-turbo', etc.
- Supports multiple providers including OpenAI direct and Azure
- Allows passing in a custom OpenAI client
- Automatically handles tool interaction request/response loops
- Has built-in image support (on compatible models)

The library is designed to make it easier to build AI applications by providing a more convenient and feature-rich interface to OpenAI's API.
Model openai/gpt-4.1 response: **Cosette** is a Python library that provides a convenient,

Cosette is a Python library that provides a high-level, convenient interface for interacting with OpenAI's API. Its primary goal is to streamline the development of AI applications by automating common patterns and managing complex workflows, while still allowing developers full control when needed.

Key features of Cosette include:

- **Stateful Conversations:** Using the `Chat()` class, it maintains conversation history, enabling more natural and persistent interactions.
- **Tool Integration:** It simplifies the process of incorporating tools into interactions by automatically handling the request and response loops required for multi-step, tool-based workflows.
- **Image Support:** Built-in capabilities for processing images are available, though this feature is limited to compatible models.
- **Model and Provider Flexibility:** Cosette supports various OpenAI models and can work with multiple providers, including direct OpenAI endpoints and Azure.
- **Custom Client Support:** Developers can supply their own OpenAI client configuration to customize settings such as the API base URL.
- **Ease of Use:** To use Cosette, you need to set the `OPENAI_API_KEY` environment variable with your OpenAI API key. Once configured, it makes building complex AI-powered applications faster and more manageable.

In essence, Cosette serves as an abstraction layer that reduces boilerplate and enhances productivity when working with OpenAI's models, especially for applications involving ongoing conversations, image processing, and tool-based workflows. For detailed documentation and guides, resources like the README and API list are available.

In [4]:
from IPython.display import Markdown
Markdown(chat.h[-1]["content"])

Cosette is a Python library that provides a high-level, convenient interface for interacting with OpenAI's API. Its primary goal is to streamline the development of AI applications by automating common patterns and managing complex workflows, while still allowing developers full control when needed.

Key features of Cosette include:

- **Stateful Conversations:** Using the `Chat()` class, it maintains conversation history, enabling more natural and persistent interactions.
- **Tool Integration:** It simplifies the process of incorporating tools into interactions by automatically handling the request and response loops required for multi-step, tool-based workflows.
- **Image Support:** Built-in capabilities for processing images are available, though this feature is limited to compatible models.
- **Model and Provider Flexibility:** Cosette supports various OpenAI models and can work with multiple providers, including direct OpenAI endpoints and Azure.
- **Custom Client Support:** Developers can supply their own OpenAI client configuration to customize settings such as the API base URL.
- **Ease of Use:** To use Cosette, you need to set the `OPENAI_API_KEY` environment variable with your OpenAI API key. Once configured, it makes building complex AI-powered applications faster and more manageable.

In essence, Cosette serves as an abstraction layer that reduces boilerplate and enhances productivity when working with OpenAI's models, especially for applications involving ongoing conversations, image processing, and tool-based workflows. For detailed documentation and guides, resources like the README and API list are available.

In [36]:

import asyncio 
from IPython.display import Markdown

async def prompt_and_merge_chat_async(prompt, chat, list_of_models, model_to_use="x-ai/grok-3-mini-beta"):
    """
    Asynchronously prompts multiple models, then merges their responses.
    Uses asyncio.to_thread to run synchronous blocking calls concurrently.
    """
    common_history = chat.h  # Synchronous access

    async def get_single_model_response(model_name_for_task, original_chat_obj, current_prompt):
        # Create a deep copy of the chat object for this task (synchronous)
        tmp_chat_branch = deepcopy(original_chat_obj)
        tmp_chat_branch.model = model_name_for_task
        
        # Run the synchronous chat call (tmp_chat_branch(current_prompt))
        # and content extraction (ot.contents) in a separate thread.
        response_content_str = await asyncio.to_thread(
            lambda p: ot.contents(tmp_chat_branch(p)), 
            current_prompt
        )
        return response_content_str

    # Create a list of tasks for asyncio.gather
    tasks = [
        get_single_model_response(model_name_in_loop, chat, prompt)
        for model_name_in_loop in list_of_models
    ]
    
    # Await all model responses concurrently
    responses_from_models = await asyncio.gather(*tasks)
    
    # Construct the prompt for the merger model
    merger_prompt_text = (
        f'You have been provided with a set of responses from various models to the latest user query: "{prompt}"\n'
        f'Your task is to synthesize these responses into a single, high-quality response. '
        f'It is crucial to critically evaluate the information provided in these responses, '
        f'recognizing that some of it may be biased or incorrect. Your response should not '
        f'simply replicate the given answers but should offer a refined, accurate, and '
        f'comprehensive reply to the instruction. Ensure your response is well-structured, '
        f'coherent, and adheres to the highest standards of accuracy and reliability.\n\n'
        f'Common conversation history (for context, but do not repeat it in your answer):\n{common_history}\n\n'
        f'Responses from models to synthesize:\n{responses_from_models}'
    )

    # The final merge call using ot.ask (synchronous, so run in a thread)
    merged_response = await asyncio.to_thread(
        ot.ask,  # The synchronous function
        merger_prompt_text,  # Argument 1 for ot.ask
        model=model_to_use  # Keyword arguments for ot.ask
    )
    
    
    # Update the history of the original chat object (synchronous)
    chat = tmp_chat_branch.h[-1] = {"role": "assistant", "content": merged_response}
    return Markdown(merged_response)

In [39]:
from onetokenpy import new_chat, Attachments
chat = new_chat(Attachments("https://answerdotai.github.io/cosette/llms.txt"))
await prompt_and_merge_chat_async("What is cosette?", chat, list_of_models)

Cosette is a term most commonly associated with a central character in Victor Hugo's iconic 1862 novel *Les Misérables*. However, it can have other meanings depending on the context. Below, I'll provide a comprehensive overview based on reliable literary sources and cultural references, ensuring the information is accurate and balanced. I'll prioritize the most prominent interpretation while briefly addressing alternatives, drawing from various perspectives while critically evaluating their relevance.

### 1. **Cosette in Literature: The Character from *Les Misérables***
Cosette, whose full name is Euphrasie, is one of the most enduring and symbolic figures in Victor Hugo's epic novel *Les Misérables*. She serves as a beacon of innocence, hope, and redemption amidst themes of social injustice, poverty, and human resilience.

- **Background and Story**: Cosette is introduced as the illegitimate daughter of Fantine, a impoverished factory worker who is forced to abandon her child to seek work. As a young girl, Cosette endures a harsh and abusive upbringing under the care of the unscrupulous innkeepers, the Thénardier family, who treat her as a servant and exploit her for labor. Her life takes a pivotal turn when she is rescued by Jean Valjean, the novel's protagonist—a former convict seeking atonement—who adopts her as his own daughter. Valjean dedicates himself to providing Cosette with a loving and secure life, shielding her from the dangers of his past. As she matures, Cosette becomes involved in a romantic subplot with Marius Pontmercy, a idealistic young revolutionary, which underscores themes of love, loss, and the promise of a better future.

- **Symbolism and Significance**: Hugo portrays Cosette as a symbol of purity and the potential for renewal in a world marred by suffering. Her journey from victimhood to happiness highlights the novel's broader critique of 19th-century French society, including class disparities and the plight of the vulnerable. This characterization has resonated deeply in adaptations, making Cosette an archetype for innocence and resilience in literature.

- **Cultural Impact**: Cosette's story has been adapted extensively across media, including stage musicals (notably the long-running Broadway production of *Les Misérables*), films (such as the 2012 movie starring Amanda Seyfried as Cosette), and television series. These portrayals have cemented her status as an iconic figure in Western culture, though interpretations can vary slightly based on the medium. For instance, while the musical emphasizes her emotional arc, film versions often highlight her visual symbolism, like her transformation from rags to more dignified attire.

This literary depiction is the most accurate and widely recognized reference, as it draws directly from Hugo's work and has been verified through historical and scholarly analysis. Other responses you've seen align on these core details, with no significant contradictions.

### 2. **Other Possible Meanings of "Cosette"**
While the literary character is the primary association, "Cosette" can have additional, less common connotations. These are worth mentioning for completeness, but they are generally derivative or minor in modern usage:

- **As a French Given Name**: Cosette is a feminine name of French origin, sometimes derived from the word "chose" (meaning "thing" or "object"). It gained popularity due to its association with Hugo's character and is occasionally used for people, pets, or even brands. For example, it might appear in contemporary contexts like baby naming trends or creative works, but it's not among the most common names today.

- **In Informal or Archaic Language**: In rare instances, "cosette" can refer to something insignificant or trivial in old French slang, akin to "a little thing" or "trifle." However, this usage is archaic and rarely encountered in everyday language. It's not a standard term in modern French or English, and sources indicate it's more of a historical curiosity than a practical definition.

Evaluations of these alternatives suggest they are less reliable as primary answers, as they stem from indirect influences of the literary character rather than independent origins. There is no evidence of "Cosette" referring to any major entities in technology, science, or other domains as of mid-2024, based on current knowledge.

### Final Thoughts
In summary, Cosette is overwhelmingly recognized as the young heroine from Victor Hugo's *Les Misérables*, embodying themes of hope and transformation that continue to inspire audiences worldwide. This synthesis prioritizes accuracy by focusing on verified historical and literary facts while acknowledging secondary meanings without overemphasizing them. If your query refers to a specific context—such as a personal name, a modern reference, or something else—please provide more details for a more tailored response. This ensures our discussion remains precise and helpful! 😊

In [40]:
chat.h

[{'role': 'user', 'content': 'What is cosette?'},
 {'role': 'assistant',
  'content': 'Cosette is a term most commonly associated with a central character in Victor Hugo\'s iconic 1862 novel *Les Misérables*. However, it can have other meanings depending on the context. Below, I\'ll provide a comprehensive overview based on reliable literary sources and cultural references, ensuring the information is accurate and balanced. I\'ll prioritize the most prominent interpretation while briefly addressing alternatives, drawing from various perspectives while critically evaluating their relevance.\n\n### 1. **Cosette in Literature: The Character from *Les Misérables***\nCosette, whose full name is Euphrasie, is one of the most enduring and symbolic figures in Victor Hugo\'s epic novel *Les Misérables*. She serves as a beacon of innocence, hope, and redemption amidst themes of social injustice, poverty, and human resilience.\n\n- **Background and Story**: Cosette is introduced as the illegitima